In [ ]:
!pip install efficientnet -q
!pip install iterative-stratification

In [ ]:
import os
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold,MultilabelStratifiedShuffleSplit

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
'''
count=0
total = 0
for i in df2.index:
    total+=1
    if '../input/public-green-notfull/png_768/'+df2.loc[i,'ID']+'.png_' not in file_list:
        count+=1
        df2.drop(index=i,inplace=True)

print('total record img:'+str(total))
print('duplicate:'+str(count))
'''

In [ ]:
'''
count=0
total = 0
for i in df2.index:
    total+=1
    if '../input/public-green-notfull/png_768/'+df2.loc[i,'ID']+'.png_' not in file_list:
        count+=1
        df2.drop(index=i,inplace=True)

print('total record img:'+str(total))
print('duplicate:'+str(count))
'''

In [ ]:
# Bug version, tensor here is graph mode, isin will cause error
'''
def map_loss(y_true,y_pred):
    lss_fn = tfa.losses.SigmoidFocalCrossEntropy()
    # y_true should be (BS,N_CLS)
    bs = y_true.shape[0]
    tp_mul = y_true * y_pred
    y_pred_sort = tf.sort(y_pred,axis=-1,direction='DESCENDING',name=None)
    #loss = tf.convert_to_tensor(np.zeros([1]),dtype=tf. float32)
    loss = tf.convert_to_tensor(np.array([]),dtype=tf. float32)
    for i in range(bs):
        arr_nonzero=[]
        nonzero=0
        arr_pred = []
        for k in tp_mul[i]:
            if k>0:
                arr_nonzero.append(k) 
                nonzero+=1
        for k in y_pred_sort[i][:nonzero]:
            arr_pred.append(k)
        print(arr_pred,arr_nonzero)
        alp = 2-np.mean(np.isin(arr_pred,arr_nonzero))
        #loss+=alp*lss_fn(y_true[i,:],y_pred[i,:])
        loss = tf.experimental.numpy.append(loss, alp*lss_fn(y_true[i,:],y_pred[i,:]), axis=None)
    return loss
'''

In [ ]:
#fixed version
def map_loss(y_true,y_pred):
    lss_fn = tfa.losses.SigmoidFocalCrossEntropy()
    # y_true should be (BS,N_CLS)
    bs = y_true.shape[0]
    tp_mul = y_true * y_pred
    y_pred_sort = tf.sort(y_pred,axis=-1,direction='DESCENDING',name=None)
    #loss = tf.convert_to_tensor(np.zeros([1]),dtype=tf. float32)
    loss = tf.convert_to_tensor(np.array([]),dtype=tf. float32)
    
    
    for i in range(bs):
        arr_nonzero=[]
        nonzero=0
        arr_pred = []
        t_pred = tf.convert_to_tensor(np.array([0]),dtype=tf. float32)
        t_nonzero = tf.convert_to_tensor(np.array([0]),dtype=tf. float32)
        for k in tp_mul[i]:
            if k>0:
                arr_nonzero.append(k) 
                nonzero+=1
                t_nonzero = t_nonzero + k
        for k in y_pred_sort[i][:nonzero]:
            arr_pred.append(k)
            t_pred = t_pred + k
        
        cor_num=0.
        total_num=0.
        if t_pred==t_nonzero:
            cor_num=1.
            total_num=1.
        else:
            cor_num=0.
            total_num=1.
        '''
        for k in range(len(arr_pred)):
            total_num+=1
            
            for u in range(len(arr_nonzero)):
                if tf.math.equal(arr_pred[k],arr_nonzero[u]):
                    cor_num+=1
                    break
        '''
        alp = 2-cor_num/total_num
        #loss+=alp*lss_fn(y_true[i,:],y_pred[i,:])
        loss = tf.experimental.numpy.append(loss, alp*lss_fn(y_true[i,:],y_pred[i,:]), axis=None)
        loss = tf.reduce_mean(loss)
    return loss

In [ ]:
def onehot(image,label):
    CLASSES = 19
    return image,tf.one_hot(label,CLASSES)
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def transform(image,label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMSIZE[IMS]
    CLASSES = 19
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4



In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment



DIM =600
n_class = 19
def _parse_image_function(example_proto,augment = True):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string)
    }
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.reshape( tf.io.decode_raw(single_example['image'],out_type=np.dtype('uint8')), (DIM,DIM, 3))
    mask =  tf.reshape(tf.io.decode_raw(single_example['label'],out_type=np.dtype('uint8')),[n_class])
    image = tf.dtypes.cast(image, tf.float32)
    mask = tf.dtypes.cast(mask, tf.float32)
    image = image/255.
    if augment: # https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu

        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_left_right(image)
            mask = tf.image.flip_left_right(mask)

        if tf.random.uniform(()) > 0.4:
            image = tf.image.flip_up_down(image)
            mask = tf.image.flip_up_down(mask)

        if tf.random.uniform(()) > 0.5:
            image = tf.image.rot90(image, k=1)
            mask = tf.image.rot90(mask, k=1)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_saturation(image, 0.7, 1.3)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_contrast(image, 0.8, 1.2)
    
    return tf.cast(image, tf.float32),tf.cast(mask, tf.float32)




def load_dataset(filenames, ordered=False, augment = False):
    AUTO = tf.data.experimental.AUTOTUNE
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image_function(ex, augment = augment), num_parallel_calls=AUTO)
    return dataset






def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True,augment_mixup_cutmix=False
                  , repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(True)
    
    AUTO = tf.data.experimental.AUTOTUNE
    #slices = paths if labels is None else (paths, labels)
    dset = load_dataset(paths)
    #dset = tf.data.Dataset.from_tensor_slices(slices)
    #dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    #dset = dset.map(transform, num_parallel_calls=AUTO) if augment else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize)
    dset = dset.map(transform,num_parallel_calls=AUTO) if augment_mixup_cutmix else dset
    dset = dset.prefetch(AUTO)
    
    return dset

In [ ]:
def count_data_items(filenames):
    img_num = 0
    for i in filenames:
        img_num +=int(i.split('_')[-2])
    return img_num

In [ ]:
#COMPETITION_NAME = "hpa-768768"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
AUG_BATCH = BATCH_SIZE
#GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)
#GCS_DS_PATH2 = KaggleDatasets().get_gcs_path('public-green-notfull')
#n_splits = 2

In [ ]:
#green
#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
#df = pd.read_csv('../input/classification-label-csv-green/df_green.csv')
#df2 = pd.read_csv('../input/atlascsv/kaggle_2021_ex_p_green_onehot.csv') declare before
#label_cols = df.columns[2:21]


#paths = GCS_DS_PATH + '/' + df['ID'] + '.png'
#labels = df[label_cols].values


#paths2 = GCS_DS_PATH2 + '/png_768/' + df2['ID'] + '.png_'
#labels2 = df2[label_cols].values

In [ ]:
#labels = labels.astype(np.float32)
#labels2 = labels2.astype(np.float32)

In [ ]:
#paths =np.concatenate((paths,paths2),axis=0)

In [ ]:
#labels = np.concatenate((labels,labels2),axis=0)

In [ ]:
'''
(
    train_paths, valid_paths, 
    train_labels, valid_labels
) = train_test_split(paths, labels, test_size=0.2, random_state=42)
'''

In [ ]:
#kf = MultilabelStratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=100)

In [ ]:
'''
for train_index, test_index in kf.split(paths, labels):
   train_paths, valid_paths = paths[train_index], paths[test_index]
   train_labels, valid_labels = labels[train_index], labels[test_index]
   break
'''

In [ ]:
# 'gs://kds-db891f03f8787fad574c283c8f45d40c0072ebcdc9e44f8b821c3788' hpatrain-green-imagetfrec
#'gs://kds-38d7c7971dfd017bc732339d9814b3cdf2211a934a6c1423e5e7a80c' hpavalid-green-imagetfrec
#'gs://kds-3867054d379fba84653fcf956ef637375f645217ab4358a185785aaf' hpatrain-ex08-green-imagetfrec
#'gs://kds-45d6c0aafb360b188d1a74670585227dbcc2847490f9dec13e879934' hpatrain-ex-916-green-imagetfrec
#'gs://kds-47b27aa31455ce4713b8c540ab3bb52d07f01b5fe2bf4be64c673821' hpatrain-ex-1724-green-imagetfrec
#'gs://kds-833ca2c0d1e0e67d9c5b3ad7c0da7c3235f418ca65031f5e6df34fc3' hpatrain-ex-2532-green-imagetfrec
# 'gs://kds-ee5eb0076b7380bde326f8d748a8103f22ed7b4f309d43ddaadb5049' hpatrain-ex-3338-green-imagetfrec

In [ ]:
train_paths=[]
train_paths.extend(tf.io.gfile.glob('gs://kds-db891f03f8787fad574c283c8f45d40c0072ebcdc9e44f8b821c3788/*'))
train_paths.extend(tf.io.gfile.glob('gs://kds-3867054d379fba84653fcf956ef637375f645217ab4358a185785aaf/*'))
train_paths.extend(tf.io.gfile.glob('gs://kds-45d6c0aafb360b188d1a74670585227dbcc2847490f9dec13e879934/*'))
train_paths.extend(tf.io.gfile.glob('gs://kds-47b27aa31455ce4713b8c540ab3bb52d07f01b5fe2bf4be64c673821/*'))
train_paths.extend(tf.io.gfile.glob('gs://kds-833ca2c0d1e0e67d9c5b3ad7c0da7c3235f418ca65031f5e6df34fc3/*'))
train_paths.extend(tf.io.gfile.glob('gs://kds-ee5eb0076b7380bde326f8d748a8103f22ed7b4f309d43ddaadb5049/*'))
valid_paths=[]
valid_paths.extend(tf.io.gfile.glob('gs://kds-f126cc8401f164a946dd39442f9eb065fe400734970516b2593a0cca/*'))

In [ ]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)
IMS = 7

decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]))
test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]))

train_dataset = build_dataset(
    train_paths,  bsize=BATCH_SIZE,cache=False, decode_fn=decoder,augment_mixup_cutmix=True
)

''' dataset checking
from tqdm import tqdm
row = 6; col = 4;
row = min(row,BATCH_SIZE//col)
for (img,label) in tqdm(train_dataset):
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break
'''

valid_dataset = build_dataset(
    valid_paths,  bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False,cache=False, augment=False
)

# test_dataset = build_dataset(
#     test_paths, cache=False, bsize=BATCH_SIZE, decode_fn=test_decoder,
#     repeat=False, shuffle=False, augment=False
# )

In [ ]:
try:
    n_labels = 19
except:
    n_labels = 1
    
with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',#tfa.losses.SigmoidFocalCrossEntropy(),
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
        
    model.summary()

In [ ]:
colour = '_green'

In [ ]:
#steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
steps_per_epoch = count_data_items(train_paths) // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'model{colour}.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=30,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{colour}.csv')